In [1]:
import os
import cv2
import random
import numpy as np
from multiprocessing import Pool
import json

In [2]:
with open('cat_dog/cat_dog_annotations.json','r') as f:
    data = json.load(f)
    print(data)

{'_via_settings': {'ui': {'annotation_editor_height': 25, 'annotation_editor_fontsize': 0.8, 'leftsidebar_width': 18, 'image_grid': {'img_height': 80, 'rshape_fill': 'none', 'rshape_fill_opacity': 0.3, 'rshape_stroke': 'yellow', 'rshape_stroke_width': 2, 'show_region_shape': True, 'show_image_policy': 'all'}, 'image': {'region_label': 'class', 'region_color': '__via_default_region_color__', 'region_label_font': '10px Sans', 'on_image_annotation_editor_placement': 'NEAR_REGION'}}, 'core': {'buffer_size': 18, 'filepath': {}, 'default_filepath': ''}, 'project': {'name': 'via_project_18Oct2022_11h49m'}}, '_via_img_metadata': {'1.jpeg45602': {'filename': '1.jpeg', 'size': 45602, 'regions': [{'shape_attributes': {'name': 'rect', 'x': 284, 'y': 27, 'width': 365, 'height': 478}, 'region_attributes': {'class': {'dog': True}}}], 'file_attributes': {}}, '2.jpeg4669': {'filename': '2.jpeg', 'size': 4669, 'regions': [{'shape_attributes': {'name': 'rect', 'x': 6, 'y': 39, 'width': 150, 'height': 107

In [3]:
classes = ['dog', 'cat']
classes_ids = {'dog': 1, 'cat': 0}

In [4]:
brightness_range = [0.7, 1.3]
noise_range = [0.05, 0.15]

In [5]:
def convert_to_yolo(image_path, via_data, yolo_path):
    img = cv2.imread(image_path)
    height, width, _= img.shape
    
    image_id = os.path.splitext(os.path.basename(image_path))[0]
    
    with open(os.path.join(yolo_path, f"{image_id}.txt"), 'w') as f:
        for region in via_data['_via_img_metadata'][image_id]['regions']:
            cetagory = list(region['region_attributes']['class'].keys()[0])
            cetagory_id = class_ids[cetagory]
            
            x = region['shape_attributes']['x']
            y = region['shape_attributes']['x']
            w = region['shape_attributes']['width']
            h = region['shape_attributes']['height']
            
            x_center = (x + w / 2) / width
            y_center = (y + h / 2) / height
            w_norm = w / width
            h_norm = h / height
            
            f.write(f"{cetagory_id} {x_center:.6f} {y_center:.6f} {w_norm:.6f} {h_norm:.6f}\n")
        for i in range(1, 6):
            brightness_factor = random.uniform(*brightness_range)
            img_bright = np.clip(img * brightness_factor, 0, 255).astype(np.uint8)
            cv2.imwrite(os.path.join(yolo_path, f"{image_id}_brightness_{i}.jpg"), img_bright)
            
            with open(os.path.join(yolo_path, f"{image_id}_brightness_{i}.txt")):
                f.write(line)
                
            noise_factor = random.uniform(*noise_range)
            img_noise = np.clip(img + noise_factor * np.random.randn(*img.shape), 0, 255).astype(np.uint8)
            cv2.imwrite(os.path.join(yolo_path, f"{image_id}_noise_{i}.jpg"), img_noise)
            
            with open(os.path.join(yolo_path, f"{image_id}_noise_{i}.txt")):
                f.write(line)

In [6]:
def augment_images(via_path, image_dir, yolo_path):
    with open(via_path, "r") as f:
        via_data = json.load(f)
        
        os.makedirs(image_dir, exist_ok=True)
        os.makedirs(yolo_path, exist_ok=True)
        
        for filename in os.listdir(image_dir):
            if not filename.lower().endswith(('.jpg', 'jpeg', 'png', 'bmp')):
                continue
            
            image_path = os.path.join(image_dir, filename)
            yolo_name = os.path.splitext(filename)[0]
            yolo_path_image = os.path.join(yolo_path, yolo_name)
            
            convert_to_yolo(via_data, image_path, yolo_path_image)
            
            img = cv2.imread(image_path)

            for i in range(1, 6):
                brightness_factor = random.uniform(*brightness_range)
                img_bright = np.clip(img * brightness_factor, 0, 255).astype(np.uint8)
                cv2.imwrite(os.path.join(image_dir, f"{yolo_name}_brightness_{i}.jpg"), img_bright)

                with open(os.path.join(yolo_path, f"{yolo_name}_brightness_{i}.txt"), 'w') as f:
                    for line in open(os.path.join(yolo_path, f"{yolo_name}.txt")):
                        f.write(line)

                noise_factor = random.uniform(*noise_range)
                img_noise = np.clip(img + noise_factor * np.random.randn(*img.shape), 0, 255).astype(np.uint8)
                cv2.imwrite(os.path.join(image_dir, f"{yolo_name}_noise_{i}.jpg"), img_noise)

                with open(os.path.join(yolo_path, f"{yolo_name}_noise_{i}.txt"), w) as f:
                    for line in open(os.path.join(yolo_path, f"{yolo_name}.txt")):
                        f.write(line)
print("Augmentation Complete")
            

Augmentation Complete
